In [82]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

In [ ]:
# BAYUABAYUAUYUAYSUYAUYAS
df = pd.read_csv('valorant_match_clsfc.csv')
df.head()
# Encode kategori
categorical_cols = ['rank', 'agent', 'map', 'outcome']
encoders = {col: LabelEncoder().fit(df[col]) for col in categorical_cols}
for col in categorical_cols:
    df[col] = encoders[col].transform(df[col])

In [84]:
features = df.drop(['game_id', 'episode', 'act', 'rank', 'date', 'outcome', 'headshot_pct', 'kills', 'assists'], axis=1)
features

,agent,map,round_wins,round_losses,deaths,kdr,avg_dmg_delta,avg_dmg,acs,num_frag
0,3,1,5,13,15,0.5,-61,76,125,3
1,3,6,4,13,15,0.2,-124,42,59,5
2,5,7,13,4,12,0.6,-46,87,132,5
3,2,1,9,13,12,1.5,23,137,230,2
4,3,5,1,13,14,0.4,-75,90,146,4
...,...,...,...,...,...,...,...,...,...,...
995,0,8,13,9,13,1.3,30,133,212,4
996,7,1,11,13,14,1.9,41,176,299,1
997,0,8,6,13,16,0.8,0,143,194,2
998,7,9,13,10,13,1.4,26,134,214,5


In [85]:
encoder = LabelEncoder()
target = encoder.fit_transform(df['outcome'])

In [86]:
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [87]:
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    target, 
    test_size=0.3, 
    random_state=42, 
    stratify=target  # Menjaga distribusi target
)

In [88]:
class SingleLayerPerceptron:
    def __init__(self, learning_rate=0.5, n_epochs=100, random_state=None):
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.random_state = random_state

    def step_function(self, z):
        return 1 if z >= 0 else 0

    def fit(self, X, y):
        # Set seed for reproducibility
        if self.random_state is not None:
            np.random.seed(self.random_state)
        
        self.weights = np.random.randn(X.shape[1])
        self.bias = 1

        for epoch in range(self.n_epochs):
            for x, y_true in zip(X, y):
                z = np.dot(x, self.weights) + self.bias
                y_pred = self.step_function(z)
                error = y_true - y_pred

                # Update weights and bias
                self.weights += self.learning_rate * error * x
                self.bias += self.learning_rate * error

    def predict(self, X):
        predictions = []
        for x in X:
            z = np.dot(x, self.weights) + self.bias
            y_pred = self.step_function(z)
            predictions.append(y_pred)
        return np.array(predictions)


In [89]:
perceptron = SingleLayerPerceptron(learning_rate=0.5, n_epochs=8,random_state=2)
perceptron.fit(X_train, y_train)

In [90]:
predictions = perceptron.predict(X_test)
accuracy = np.mean(predictions == y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Weights:", perceptron.weights)

Accuracy: 99.67%
Weights: [ 0.56248461 -0.69568138  6.94582012 -7.23058591 -0.18161515  1.36708575
  1.16013751 -1.74269712 -0.29704758 -0.54336163]


In [91]:
# Konversi nilai numerik menjadi kategori 'win' dan 'loss'
results = pd.DataFrame({
    'Actual': encoder.inverse_transform(y_test),
    'Predicted': encoder.inverse_transform(predictions)
})

# Mapping 1 ke 'win' dan 0 ke 'loss'
mapping = {1: 'win', 0: 'lose'}
results['Actual'] = results['Actual'].map(mapping)
results['Predicted'] = results['Predicted'].map(mapping)

# Simpan hasil ke file CSV
results.to_csv('slp_step_func_results_y_test.csv', index=False)


In [92]:
predictions = perceptron.predict(X_train)
accuracy = np.mean(predictions == y_train)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Weights:", perceptron.weights)
print("Bias:", perceptron.bias)

Accuracy: 99.86%
Weights: [ 0.56248461 -0.69568138  6.94582012 -7.23058591 -0.18161515  1.36708575
  1.16013751 -1.74269712 -0.29704758 -0.54336163]
Bias: 1.0


In [93]:
# Konversi nilai numerik menjadi kategori 'win' dan 'loss'
results = pd.DataFrame({
    'Actual': encoder.inverse_transform(y_train),
    'Predicted': encoder.inverse_transform(predictions)
})

# Mapping 1 ke 'win' dan 0 ke 'loss'
mapping = {1: 'win', 0: 'lose'}
results['Actual'] = results['Actual'].map(mapping)
results['Predicted'] = results['Predicted'].map(mapping)

# Simpan hasil ke file CSV
results.to_csv('slp_step_func_results_y_train.csv', index=False)
